In [1]:
import sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from utils import *


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
SUMO_HOME found
Using device: cuda


In [2]:
planned_path = get_planned_path()
path_values = list(planned_path.values())
path_values = torch.tensor(path_values).float() / 10

 Retrying in 1 seconds
***Starting server on port 39695 ***
Loading net-file from './config/osm.net.xml.gz' ... done (62ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (171ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.


In [ ]:
starts = checkpoints[:, :-1, :]
ends = checkpoints[:, 1:, :]
starts_ends = ends - starts
norms = torch.norm(starts_ends, dim=2, keepdim=True)
print(norms.max())

In [3]:
def project_to_nearest(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        return projections, min_indices


def project_to_nearest_with_checkpoints(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        next_checkpoint = to[range(to.shape[0]), min_indices]
        pad_to = F.pad(to, (0, 0, 1, 0), value=0)
        next_next_checkpoint = pad_to[range(to.shape[0]), min_indices + 1]
        projections_with_checkpoints = torch.cat((projections, next_checkpoint, next_next_checkpoint), dim=1)
        return projections_with_checkpoints

In [4]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list).float() / 10

next_checkpoint = torch.zeros_like(all_trajectories_tensor)
next_next_checkpoint = torch.zeros_like(all_trajectories_tensor)
path_values_pad_1 = F.pad(path_values, (0, 0, 1, 0))
for i in range(all_trajectories_tensor.shape[1]):
    _, min_indices = project_to_nearest(all_trajectories_tensor[:, i], path_values_pad_1)
    next_checkpoint[:, i] = path_values_pad_1[range(path_values_pad_1.shape[0]), min_indices+1]
    next_next_checkpoint[:, i] = path_values_pad_1[range(path_values_pad_1.shape[0]), min_indices+2]

all_trajectories_tensor = torch.cat((all_trajectories_tensor, next_checkpoint, next_next_checkpoint), dim=2)

def add_noise(sequence, path, noise_level=500):
    noise = torch.rand(1) * noise_level
    return sequence + noise, path + noise

# Apply noise augmentation to the dataset
# def augment_data(trajectory, paths, num_augmentations=5, noise_level=500):
#     augmented_trajectory = []
#     augmented_paths = []
    
#     for _ in range(num_augmentations):
#         for i in range(len(trajectory)):
#             sequence = trajectory[i]
#             path = paths[i]
#             augmented_sequence, augmented_path = add_noise(sequence, path, noise_level)
#             augmented_trajectory.append(augmented_sequence)
#             augmented_paths.append(augmented_path)

#     return torch.stack(augmented_trajectory), torch.stack(augmented_paths)

# all_trajectories_tensor, path_values = augment_data(all_trajectories_tensor, path_values)   

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) < p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.0, max_mask_ratio=0.1, missing_ratio=0.6, complete_traj_ratio=0.75):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        if np.random.rand() < complete_traj_ratio:
            masks.append(torch.zeros_like(initial_mask).tolist())
            continue
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 0
        mask[1] = 0
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = path_values[:train_size]
val_path_values = path_values[train_size:]

train_mask = generate_masks(train_trajectories_tensor)

class DatasetWithPlans(Dataset):
    def __init__(self, tensor, input_mask, path_values):
        self.tensor = tensor.float().to(device)
        self.input_mask = input_mask.float().to(device)
        self.path_values = path_values.float().to(device)
        self.num = self.path_values.shape[0]
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx], self.input_mask[idx], self.path_values[idx % self.num]

train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_path_values)

In [ ]:
# find the zero ratio in the dataset
torch.mean((train_trajectories_tensor == 0).float())

tensor(0.2177)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        # out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        mse = torch.mean(((output - target) ** 2) * mask)
        directional_diff = torch.mean(torch.abs(torch.atan2(output[:, 1], output[:, 0]) - torch.atan2(target[:, 1], target[:, 0])))
        distance_diff = torch.mean(torch.norm(output, dim=1)-torch.norm(target, dim=1))
        return mse + directional_diff + distance_diff

In [ ]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            # steps = 0
            for t in range(1, seq_len):
                # new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
                # if new_steps == 0:
                #     break
                prediction, hidden = model(current_input, hidden)
                # steps += new_steps
                previous_input = inputs[:, t-1, :2]
                if epoch > 25:
                    projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, planned_path)
                    current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                if t > 1:
                    loss += criterion(prediction - previous_input, inputs[:, t, :2]-previous_input) #(current_input-previous_input).squeeze(1))
            # loss /= steps
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=6, hidden_size=256, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 60, optimizer, criterion)

Epoch 1, Loss: 1330690.340625
Epoch 2, Loss: 884168.13125
Epoch 3, Loss: 410830.6796875
Epoch 4, Loss: 166788.5236328125
Epoch 5, Loss: 96055.437890625
Epoch 6, Loss: 71951.233984375
Epoch 7, Loss: 59056.58359375
Epoch 8, Loss: 41547.1787109375
Epoch 9, Loss: 26173.12861328125
Epoch 10, Loss: 16954.83642578125
Epoch 11, Loss: 12295.457568359376
Epoch 12, Loss: 10411.494140625
Epoch 13, Loss: 8958.462768554688
Epoch 14, Loss: 8451.895922851563
Epoch 15, Loss: 8039.536645507813
Epoch 16, Loss: 7704.617993164063
Epoch 17, Loss: 8093.477978515625
Epoch 18, Loss: 7824.77841796875
Epoch 19, Loss: 7714.673681640625
Epoch 20, Loss: 7512.333325195313
Epoch 21, Loss: 7321.022485351563
Epoch 22, Loss: 7282.075024414063
Epoch 23, Loss: 7468.293530273438
Epoch 24, Loss: 7356.778466796875
Epoch 25, Loss: 7157.15810546875
Epoch 26, Loss: 7438.85234375
Epoch 27, Loss: 8194.4119140625
Epoch 28, Loss: 7582.842529296875
Epoch 29, Loss: 7673.1119140625
Epoch 30, Loss: 7523.202392578125
Epoch 31, Loss: 779

In [32]:
# def checkModel(model, inputs, masks, paths):
#     model.eval()
#     with torch.no_grad():
#         hidden = None
#         seq_len = inputs.size(1)
#         current_input = inputs[:, 0, :].unsqueeze(1)
#         for t in range(1, seq_len):
#             prediction, hidden = model(current_input, hidden)
#             projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
#             current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
#             print(projection_with_checkpoints[0][:2]*10, inputs[:, t, :][0][:2]*10)

# # get the first batch in dataloader
# val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
# val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
# val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
# inputs, masks, paths = next(iter(val_dataloader))

# checkModel(model, inputs, masks, paths)

tensor([3981.5149, 3950.8906], device='cuda:0') tensor([5331.3096, 4042.4995], device='cuda:0')
tensor([5293.6890, 4087.3218], device='cuda:0') tensor([5333.3589, 4039.8730], device='cuda:0')
tensor([5354.9678, 4012.1804], device='cuda:0') tensor([5336.5205, 4035.8215], device='cuda:0')
tensor([5351.6748, 4016.4004], device='cuda:0') tensor([5340.9448, 4030.1516], device='cuda:0')
tensor([5348.5967, 4020.3447], device='cuda:0') tensor([5346.6807, 4022.8003], device='cuda:0')
tensor([5347.5508, 4016.4939], device='cuda:0') tensor([5352.3267, 4015.5649], device='cuda:0')
tensor([5348.5347, 4015.2329], device='cuda:0') tensor([5357.9546, 4008.3525], device='cuda:0')
tensor([5349.6064, 4013.8596], device='cuda:0') tensor([5363.2852, 4001.5225], device='cuda:0')
tensor([5357.1260, 4004.2249], device='cuda:0') tensor([5368.4058, 3994.9607], device='cuda:0')
tensor([5359.6489, 4000.9917], device='cuda:0') tensor([5373.8208, 3988.0225], device='cuda:0')
tensor([5363.1763, 3996.4702], device='c

In [14]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projection_with_checkpoints[:, :2] - inputs[:, t, :2]) * 10, dim=1) * (inputs[:, t, :2] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

# model = GRUModel(input_size=6, hidden_size=128, num_layers=2, num_classes=2).to(device)
# model.load_state_dict(torch.load('gru_model_e.pth'))
for i in [0.9, 0.8, 0.7, 0.6, 0.5, 0]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 52.15977427728795
Validation Loss: 30.456816802863198
Validation Loss: 22.356280817025592
Validation Loss: 18.26728633528924
Validation Loss: 15.854126025314123
Validation Loss: 10.846940906193359


In [15]:
# Save the model
torch.save(model.state_dict(), 'gru_model.pth')